In [1]:
import sys
from datasets import load_dataset
import polars as pl
from IPython.display import Markdown
import tiktoken
sys.path.append("..")
from synthetic_data.tasks import _process_gutenberg_row
%load_ext autoreload
%autoreload 2

/home/brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dataset = load_dataset(
    "sam-paech/gutenberg3-generalfiction-scifi-fantasy-romance-adventure-dpo",
    trust_remote_code=True,
)["train"]
dataset_pl: pl.DataFrame = dataset.to_polars()
tiktoken_encoder = tiktoken.get_encoding("o200k_base")

In [ ]:
lengths = []
for i, row in enumerate(dataset_pl.head(10).iter_rows(named=True)):
    source = row["source"]
    display(Markdown(f"### Sample {i} - {source}"))
    row_processed = _process_gutenberg_row(row,tiktoken_encoder)
    display(Markdown(f"#### Original ({row_processed['encoded_length']} tokens) \n {row_processed['text']}"))

In [2]:
formatted_pq = pl.read_parquet("../screenplay_scenes_summarized_full.parquet")

In [5]:
formatted_pq.shape

(10256, 9)

In [6]:
from synthetic_data.tasks import Output, SceneElementType
for row in formatted_pq[1000:1010].iter_rows(named=True):
    output_obj = Output.model_validate_json(row["output"])
    for element in output_obj.items:
        fmt_str = f"{element.type.name} - {element.character}"
        if element.type == SceneElementType.DIALOGUE:
            fmt_str = f"**{element.character}**: {element.content}"
        elif element.type == SceneElementType.ACTION:
            fmt_str = f"*{element.content}*"
        else:
            fmt_str = f"{element.type.name} - {element.content}"
        display(Markdown(fmt_str))

SCENE_HEADING - MATOACA CITY, West Virginia, October 16, 1884

**Narrator**: DEAREST, DEAREST MOTHER:

*sits down to write*

**Narrator**: We got here this morning after a dreadful trip--nine or ten hours late--and this is the first minute I've had when I could sit down and write to you.

**Narrator**: All the way on the train I was thinking of you and dear father, and longing for you so that I could hardly keep back the tears.

...

**Marquis**: "It rejoices me," the Marquis returned, "that a conspirator of many years' standing should commend my maiden effort."

*He rose.*

**John Bulmer**: "Well,--no!" said he, at last; "I commend your cleverness, Monsieur de Soyecourt, but as concerns your hand I must confess to a distaste."

**Marquis**: "Because at the bottom of your heart you despise me," he said. "Ah, believe me, monsieur, your contempt for de Soyecourt is less great than mine. And yet I have a weakness for him,--a weakness which induces me to indulge all his desires."

*He bowed with ceremony and left the garden.*